# This code is an implementation of Algorithm 2 in the paper:
T. Le, V. Subramanian, R. Berry, **Quantifying the Utility of Noisy Reviews in Stopping Information Cascades**, manuscript submitted to IEEE CDC, 2016. Archived version available at: https://archive.org/details/CDC2016Archive

## The algorithm calculates, for the case of bad product, the probability of the time until a correct information cascade happens (i.e., people learn the true value of the product and stop buying it), $P[\tau = n|h_1]$. Here we are conditioning on $h_1$, the state of the system after the first agent's action choice and his review.

## The algorithm idea is to build a breadth-first tree conditioned on $h_1$.

## Here, n is the first agent who starts the correct cascade.
Computational power of my laptop allows up to around 17.

## The outputs are written to .csv files for different values of the private signal $p$. Then these results are read to MATLAB for further processing and generatting the plots.

In [36]:
import numpy as np
from pandas import Series, DataFrame # since we use Series, DataFrame so often in this course
import pandas as pd

import math

In [14]:
# build a from a list of dictionary instead of a dictionary with values being lists
# each dictionary is a node with all the attributes
# 'Node': name of the node, also the realization at that node

root1 = {'Node': 'L_', 'level': 1, 'a': -1, 'r': 0,  'hi': float('inf'), 'lo': 0, 'min' : -1, 'max' : -1,
         'stat': 'notdone', 'flag': 'fail'}
tree1 = [root1]
tree1_frame = DataFrame(tree1)
#Show
tree1_frame

,Node,a,flag,hi,level,lo,max,min,r,stat
0,L_,-1,fail,inf,1,0,-1,-1,0,notdone


In [13]:
root2 = {'Node': 'HB', 'level': 1, 'a': +1, 'r': -1, 'hi': float('inf'), 'lo': 1/2, 'min' : -1, 'max' : 1,
         'stat': 'notdone', 'flag': 'fail'}
tree2 = [root2]
tree2_frame = DataFrame(tree2)
#Show
tree2_frame

,Node,a,flag,hi,level,lo,max,min,r,stat
0,HB,1,fail,inf,1,0.5,1,-1,-1,notdone


In [15]:
root3 = {'Node': 'HG', 'level': 1, 'a': +1, 'r': +1, 'hi': float('inf'), 'lo': 0, 'min' : 1, 'max' : float('inf'),
         'stat': 'notdone', 'flag': 'fail'}
tree3 = [root3]
tree3_frame = DataFrame(tree3)
#Show
tree3_frame

,Node,a,flag,hi,level,lo,max,min,r,stat
0,HG,1,fail,inf,1,0,inf,1,1,notdone


In [10]:
def eval_h(a, r, x):
    if r==0:
        return a
    elif x==0 and r>0:
        return float('inf')
    elif x==0 and r<0:
        return -float('inf')
    else:
        return a + r/x 

In [31]:
# Test cases
eval_h(1,-1,-float('inf'))

1.0

In [213]:
print(eval_h(-1,-1,1)==-1)

False


In [11]:
def new_minmax(new_a, new_r, cur_lo, cur_hi): # always cur_lo < cur_hi
    '''
    return new_min, new_max evaluated with new_a, new_r at x = cur_lo and x = cur_hi
    basically just determin which value of x among the two gives the higher value of h
    the higher value is new_max, the lower value is new_min
    '''
    if new_r == 0:
        # new_min = new_a, new_max = new_a
        return new_a, new_a # note that this is a tuple.
    elif new_r > 0:
        # new_min = eval_h(new_a, new_r, cur_hi), new_max = eval_h(new_a, new_r, cur_lo)
        return eval_h(new_a, new_r, cur_hi), eval_h(new_a, new_r, cur_lo)
    else:
        return eval_h(new_a, new_r, cur_lo), eval_h(new_a, new_r, cur_hi)
    # new_min = new_max in only 1 cases new_r ==0 
    # the case cur_lo = cur_hi cannot happen 
    # cur_lo can not be <0
    
        

In [87]:
# Test cases
# new_minmax(-2,0,0,float('inf'))
# new_minmax(0,-1,0,float('inf'))
# new_minmax(0,1,0,float('inf'))
# new_minmax(1,-2,1,float('inf'))
# new_minmax(1,1,1/2,1)
# new_minmax(1,1,1/2,1/2)
# new_minmax(1,1,0,float('inf'))

(1.0, inf)

In [13]:
def int_update(new_a, new_r, cur_lo, cur_hi):
    # to return 3 lists, each specifies the condition for new_h to be in 3 corresponding intervals:
    # (-inf, -1), [-1, 1], (1,+inf)
    
    new_min, new_max = new_minmax(new_a, new_r, cur_lo, cur_hi)
    # Now compare new_min, new_max with cur_min, cur_max to set the new intervals
    if new_min == new_max: # then no change in cond, now we only return the corresponding interval, the other two are empty
        if new_min < -1:
            return [cur_lo, cur_hi], [], []
        elif new_min > 1:
            return [], [], [cur_lo, cur_hi]
        else:
            return [], [cur_lo, cur_hi], []
    else: #i.e. new_min < new_max (also mean new_r != 0)
        # now, we need to find the new conditions: new_lo, new_h for each of the 3 intervals
        # case 1: new_max < -1
        if new_max <= -1:
            return [cur_lo, cur_hi], [], []
        elif new_min >= 1:
            return [], [], [cur_lo, cur_hi]
        elif new_min >= -1 and new_max <= 1:
            return [], [cur_lo, cur_hi], []
        elif new_min <-1 and new_max >-1 and new_max <= 1:        
            # solve for new_h = -11
            sol_m1 = new_r/(-new_a - 1) # we must have cur_lo < sol_m1 < cur_hi, since new_h is linear, monotonic
            if new_r > 0:
                return [sol_m1, cur_hi], [cur_lo, sol_m1], []
            elif new_r < 0:
                return [cur_lo, sol_m1], [sol_m1, cur_hi], []
        elif new_min >= -1 and new_min < 1 and new_max >= 1:
            sol_p1 = new_r/(-new_a + 1) # we must have cur_lo < sol_m1 < cur_hi, since new_h is linear, monotonic
            if new_r > 0:
                return [], [sol_p1, cur_hi], [cur_lo, sol_p1]
            elif new_r < 0:
                return [], [cur_lo, sol_p1], [sol_p1, cur_hi]
        elif new_min <-1 and new_max >1:
            sol_m1 = new_r/(-new_a - 1)
            sol_p1 = new_r/(-new_a + 1)
            if new_r > 0:
                return [sol_m1, cur_hi], [sol_p1, sol_m1], [cur_lo, sol_p1]
            elif new_r < 0:
                return [cur_lo, sol_m1], [sol_m1, sol_p1], [sol_p1, cur_hi]

In [81]:
# Test cases, for n=4
#--------For h1 = -1
int_update(-2, 0, 0, float('inf')) # ([0, inf], [], [])

#--------For h1 = 1-1/x
int_update(2, -2, 1/2, float('inf')) # ([0.5, 0.6666666666666666], [0.6666666666666666, 2.0], [2.0, inf])
int_update(0, -1, 1/2, float('inf')) # ([0.5, 1.0], [1.0, inf], [])
int_update(2, 0, 1/2, float('inf')) # ([], [], [0.5, inf])
int_update(-1, -1, 1, float('inf')) # ([1, inf], [], [])
int_update(1, -2, 1, float('inf')) # ([], [1, inf], [])
int_update(1, 0, 1, float('inf')) # ([], [1, inf], [])
int_update(0, -2, 1, float('inf')) #([1, 2.0], [2.0, inf], [])
int_update(2, -3, 1, float('inf')) # ([], [1, 3.0], [3.0, inf])
int_update(2, -1, 1, float('inf')) # ([], [], [1, inf])
int_update(0, 0, 1, float('inf')) # ([], [1, inf], [])
int_update(2, -1, 1, float('inf')) # ([], [], [1, inf])
int_update(2, 1, 1, float('inf')) # ([], [], [1, inf])
int_update(2, -3, 2, float('inf')) # ([], [2, 3.0], [3.0, inf])
int_update(2, -4, 3, float('inf')) # ([], [3, 4.0], [4.0, inf])
int_update(1, -3, 2, 3) # ([], [2, 3], [])
int_update(3, -1, 2, 3) # ([], [], [2, 3])
int_update(3, -3, 2, 3) # ([], [], [2, 3])
int_update(3, -1, 2, float('inf')) # ([], [], [2, inf])
int_update(1, -2, 2/3, 2) # ([0.6666666666666666, 1.0], [1.0, 2], [])
int_update(0, -2, 1, 2) # ([1, 2], [], [])
int_update(2, -3, 1, 2) # ([], [1, 2], [])
int_update(2, -1, 1, 2) # ([], [], [1, 2])
#--------For h1 = 1+1/x 

([], [], [1, 2])

In [14]:
def create_node(new_name, new_a, new_r, cur_lo, cur_hi):
    '''
    Take the new_a, new_r and create the new node
    '''
    new_min, new_max = new_minmax(new_a, new_r, cur_lo, cur_hi)
    return {'node': new_name, 'a': new_a, 'r': new_r,  'hi': cur_hi, 'lo': cur_lo, 
                                    'min' : new_min, 'max' : new_max}

In [130]:
# import bisect 
def BFS_tree(n, root):
    '''
    n: max of tau
    3 possibilities: h1 = -1 (a=-1,r=0), h1 = 1-1/x (a=1,r=-1), h1 = 1+1/x (a=1,r=1)
    '''
    #tree = [] # initialize the list of list of dictionaries, each dictionary is a node
    # each list of dictionaries is the node at a certain level. Beware of the order, if not need to set the first element
    # of the list as the indicator of level.
    
    tree = []
    for i in range(n):
        tree.append([])
    tree[0] = [root]

    #tree1_frame = DataFrame(tree1)
    qualified = []
    intervals = []
    
    cur_height = 0;
    
    while (cur_height < n): # tree[cur_height] is the list of nodes (dicts) at the current height
        for i, node in enumerate(tree[cur_height]):
            # node = tree[cur_height][0]
            # node is a dictionary
            
            if cur_height < n-1: # not a leaf, consider adding children
                if (node['min'] < -1): # i.e N cascade
                    next
                elif (node['max'] > 1): # i.e. Y cascade
                    # --------------------------then 2 possibility to add, no Signal, only Reviews B or G
                    
                    # -- 1st possibility: : new_a = cur_a, new_r = cur_r - 1, R = B
                    new_a = node['a']
                    new_r = node['r'] - 1 
                    new_name = node['node'] + '_B' # the underscore is to denote missing signal
                    int_below_m1, int_m1_p1, int_above_p1 = int_update(new_a, new_r, node['lo'], node['hi'])
                    
                    if ((not not int_below_m1) and (cur_height == n-2)):
                        #i.e. N cascade exists for its children, but the children are leaf, so need to add this
                        new_node = create_node(new_name, new_a, new_r, int_below_m1[0], int_below_m1[1])
                        tree[cur_height+1].append(new_node)
                    if (not not int_m1_p1):
                        new_node = create_node(new_name, new_a, new_r, int_m1_p1[0], int_m1_p1[1])
                        tree[cur_height+1].append(new_node)
                    if (not not int_above_p1):
                        new_node = create_node(new_name, new_a, new_r, int_above_p1[0], int_above_p1[1])
                        tree[cur_height+1].append(new_node)
                    
                    # -- 2nd possibility: : new_a = cur_a, new_r = cur_r + 1, R = G
                    new_a = node['a']
                    new_r = node['r'] + 1 
                    new_name = node['node'] + '_G' 
                    int_below_m1, int_m1_p1, int_above_p1 = int_update(new_a, new_r, node['lo'], node['hi'])
                    
                    if ((not not int_below_m1) and (cur_height == n-2)):
                        #i.e. N cascade exists for its children, but the children are leaf, so need to add this
                        new_node = create_node(new_name, new_a, new_r, int_below_m1[0], int_below_m1[1])
                        tree[cur_height+1].append(new_node)
                    if (not not int_m1_p1):
                        new_node = create_node(new_name, new_a, new_r, int_m1_p1[0], int_m1_p1[1])
                        tree[cur_height+1].append(new_node)
                    if (not not int_above_p1):
                        new_node = create_node(new_name, new_a, new_r, int_above_p1[0], int_above_p1[1])
                        tree[cur_height+1].append(new_node)
                else: # i.e. node['min'] >= -1 and node['max'] <= 1, then agent use Signal
                    # --------------------------then 3 possibility to add: L , HB, HG
                    
                    # -- 1st possibility: : new_a = cur_a - 1, new_r = cur_r (since action = N, no review)
                    new_a = node['a'] - 1
                    new_r = node['r'] 
                    new_name = node['node'] + 'L_' # the underscore is to denote missing review
                    int_below_m1, int_m1_p1, int_above_p1 = int_update(new_a, new_r, node['lo'], node['hi'])
                    #print(int_below_m1, int_m1_p1, int_above_p1) # [0, inf] [] []
                    
                    if ((not not int_below_m1) and (cur_height == n-2)):
                        #i.e. N cascade exists for its children, but the children are leaf, so need to add this
                        new_node = create_node(new_name, new_a, new_r, int_below_m1[0], int_below_m1[1])
                        tree[cur_height+1].append(new_node)
                    if (not not int_m1_p1):
                        new_node = create_node(new_name, new_a, new_r, int_m1_p1[0], int_m1_p1[1])
                        tree[cur_height+1].append(new_node)
                    if (not not int_above_p1):
                        new_node = create_node(new_name, new_a, new_r, int_above_p1[0], int_above_p1[1])
                        tree[cur_height+1].append(new_node)
                        
                    # -- 2nd possibility: : new_a = cur_a + 1, new_r = cur_r - 1
                    new_a = node['a'] + 1
                    new_r = node['r'] - 1
                    new_name = node['node'] + 'HB'
                    int_below_m1, int_m1_p1, int_above_p1 = int_update(new_a, new_r, node['lo'], node['hi'])
                    
                    if ((not not int_below_m1) and (cur_height == n-2)):
                        #i.e. N cascade exists for its children, but the children are leaf, so need to add this
                        new_node = create_node(new_name, new_a, new_r, int_below_m1[0], int_below_m1[1])
                        tree[cur_height+1].append(new_node)
                    if (not not int_m1_p1):
                        new_node = create_node(new_name, new_a, new_r, int_m1_p1[0], int_m1_p1[1])
                        tree[cur_height+1].append(new_node)
                    if (not not int_above_p1):
                        new_node = create_node(new_name, new_a, new_r, int_above_p1[0], int_above_p1[1])
                        tree[cur_height+1].append(new_node)
                        
                    # -- 3rd possibility: : new_a = cur_a + 1, new_r = cur_r + 1
                    new_a = node['a'] + 1
                    new_r = node['r'] + 1
                    new_name = node['node'] + 'HG'
                    int_below_m1, int_m1_p1, int_above_p1 = int_update(new_a, new_r, node['lo'], node['hi'])
                    
                    if ((not not int_below_m1) and (cur_height == n-2)):
                        #i.e. N cascade exists for its children, but the children are leaf, so need to add this
                        new_node = create_node(new_name, new_a, new_r, int_below_m1[0], int_below_m1[1])
                        tree[cur_height+1].append(new_node)
                    if (not not int_m1_p1):
                        new_node = create_node(new_name, new_a, new_r, int_m1_p1[0], int_m1_p1[1])
                        tree[cur_height+1].append(new_node)
                    if (not not int_above_p1):
                        new_node = create_node(new_name, new_a, new_r, int_above_p1[0], int_above_p1[1])
                        tree[cur_height+1].append(new_node)
                           
            else: # node is a leaf, consider adding to qualified list
                if (node['min'] < -1): # i.e N cascade
                    qualified.append(node)
                    # bisect.insort(intervals, node['lo']) 
                    # bisect.insort(intervals, node['hi'])
                    intervals.extend([node['lo'], node['hi']])
                else: # a leaf that doesn't creat a N cascade, just ignore it.
                    next
                        
        cur_height = cur_height + 1
    intervals = sorted(list(set(intervals)))
    return qualified, intervals
    

In [47]:
root1 = {'node': 'L_', 'a': -1, 'r': 0,  'hi': float('inf'), 'lo': 0, 'min' : -1, 'max' : -1}
root2 = {'node': 'HB', 'a': +1, 'r': -1, 'hi': float('inf'), 'lo': 1/2, 'min' : -1, 'max' : 1}
root3 = {'node': 'HG', 'a': +1, 'r': +1, 'hi': float('inf'), 'lo': 0, 'min' : 1, 'max' : float('inf')}

In [135]:
n=2
N_leaf, intervals = BFS_tree(n, root1)
tree_N_leaf = DataFrame(N_leaf)
#print(tree_built)
#print(N_leaf)
#print(len(tree_built))
#tree_level0 = DataFrame(tree_built[0])
#tree_level1 = DataFrame(tree_built[1])
#tree_level2 = DataFrame(tree_built[2])
#tree_level3 = DataFrame(tree_built[3])

In [136]:
intervals

[0, 1.0, inf]

In [137]:
tree_N_leaf

,a,hi,lo,max,min,node,r
0,-2,inf,0,-2,-2.000000,L_L_,0
1,0,1.000000,0,-1,-inf,L_HB,-1


In [232]:
def probability(qualified, intervals, p, d_list):
    '''
    return the probability P[tau=n] for each x in x_list
    qualified is the list of nodes (dictionaries) that are leaves and lead to N cascade
    '''
    q = 1 - p
    d1_list = [1- d for d in d_list] # this is 1 - d_list
    base = [a/b for a,b in zip(d1_list, d_list)]
    
    x_list = [math.log(q/p, d) for d in base]
    #print(x_list)
    
    prob_arr = np.zeros(len(d_list)) # initialize the array of
    # prob of N cascade for each value of delta in d_list
    # we will go through each qualified nodes and check which values
    # of x in [lo, hi] of that node and then add the probability of that node to
    # all the element of this array that has x falls into [lo, hi]
    
    for j, node in enumerate(qualified):
        name = node['node']
        num_L = name[2:].count('L') # slice [2:] since we need to ignore agent 1's information
        num_H = name[2:].count('H')
        num_B = name[2:].count('B')
        num_G = name[2:].count('G')
        num_L_all = name.count('L')
        num_H_all = name.count('H')
        num_B_all = name.count('B')
        num_G_all = name.count('G')
        cur_a = num_H_all - num_L_all
        cur_r = num_G_all - num_B_all
        
        for i, x in enumerate(x_list):
            if ((x >= node['lo']-1.0e-14) and (x <= node['hi']+1.0e-14)):
                # Note that V=0, so L and B correspond to higher probabilities: p and delta
                delta = d_list[i]
                delta1 = d1_list[i] # 1-delta
                
                if ((abs(p-delta) < 1.0e-15) and ( eval_h(cur_a, cur_r, 1) == -1 )):
                    next
                else:
                    prob_arr[i] = prob_arr[i] + (p**num_L) * (q**num_H) * (delta1**num_G) * (delta**num_B)

    return prob_arr

In [285]:
n=17
#N_leaf, intervals = BFS_tree(n, root1)
#print(len(N_leaf))
# gives 95898 combinations that lead to N cascade
#N_leaf, intervals = BFS_tree(n, root2)
#print(len(N_leaf))
# gives 172840 combinations that lead to N cascade
N_leaf, intervals = BFS_tree(n, root3)
print(len(N_leaf))
# gives 121496 combinations that lead to N cascade

121496


In [252]:
n=17
N_leaf, intervals = BFS_tree(n, root1)
p = 0.7
q = 1 - p
#d_list = np.arange(0.6,1,0.1)
d_list = np.arange(0.51,1,0.01)
d1_list = [1- d for d in d_list] # this is 1 - d_list
base = [a/b for a,b in zip(d1_list, d_list)]   
x_list = [math.log(q/p, d) for d in base]
arr = probability(N_leaf, intervals, p, d_list)

In [227]:
print(d_list[19])
print(p)
print(d_list[19]==p)
print(d_list[19] - p)
print(abs(d_list[19] - p) < 1.0e-15)
print(x_list[19] >= 1)
print(x_list[19] - 1)

# need to use abs()< tol to check for equality of 2 float values

0.7
0.7
False
2.22044604925e-16
True
False
-1.2212453270876722e-15


In [221]:
len(arr)

49

In [251]:
np.savetxt("foo.csv", arr)

In [253]:
DataFrame(N_leaf)

,a,hi,lo,max,min,node,r
0,0,2.000000,1,-1,-2.000000,L_HBHBL_,-2
1,-2,inf,1,-1,-2.000000,L_HGL_L_,1
2,1,0.500000,0,-1,-inf,L_HG_BHB,-1


In [254]:
def main(n, p, d_list):
    '''
    n: max of tau
    call the BSF 3 times for 3 different possibilities: h1 = -1 (a=-1,r=0), h1 = 1-1/x (a=1,r=-1), h1 = 1+1/x (a=1,r=1)
    
    '''
    # Initialize the 3 trees
    root1 = {'node': 'L_', 'a': -1, 'r': 0,  'hi': float('inf'), 'lo': 0, 'min' : -1, 'max' : -1}
    #tree1 = [[root1]]
    # tree1_frame = DataFrame(tree1)
    
    root2 = {'node': 'HB', 'a': +1, 'r': -1, 'hi': float('inf'), 'lo': 1/2, 'min' : -1, 'max' : 1}    
    root3 = {'node': 'HG', 'a': +1, 'r': +1, 'hi': float('inf'), 'lo': 0, 'min' : 1, 'max' : float('inf')}
    
    N_leaves_L, intervals_L = BFS_tree(n, root1) 
    # list of leaves, each elements is a dictionary (a node)
    # intervals are a list of intervals of x that correspond to conditions for x of all the nodes in N_leaves_L
    # only use the node (i.e name, or realization ), and the interval [lo, hi]
    
    N_leaves_HB, intervals_HB = BFS_tree(n, root2)
    N_leaves_HG, intervals_HG = BFS_tree(n, root3)
    
    
    prob_L = probability(N_leaves_L, intervals_L, p, d_list) # numpy array
    prob_HB = probability(N_leaves_HB, intervals_HB, p, d_list) # numpy array
    prob_HG = probability(N_leaves_HG, intervals_HG, p, d_list) # numpy array
    
    return prob_L, prob_HB, prob_HG # the 3 arrays of probabilities for L, HB, HG

In [273]:
n=1 # need to work on this special case
prob_L1, prob_HB1, prob_HG1 = main(n, p, d_list)
np.savetxt("fooL1.csv", prob_L1)
np.savetxt("fooHB1.csv", prob_HB1)
np.savetxt("fooHG1.csv", prob_HG1)

In [272]:
n=2
prob_L2, prob_HB2, prob_HG2 = main(n, p, d_list)
np.savetxt("fooL2.csv", prob_L2)
np.savetxt("fooHB2.csv", prob_HB2)
np.savetxt("fooHG2.csv", prob_HG2)

In [264]:
n=3
p = 0.7
d_list = np.arange(0.501,1,0.001)
prob_L3, prob_HB3, prob_HG3 = main(n, p, d_list)
# write the 3 list to file
np.savetxt("fooL3.csv", prob_L3)
np.savetxt("fooHB3.csv", prob_HB3)
np.savetxt("fooHG3.csv", prob_HG3)

In [265]:
n=4
prob_L4, prob_HB4, prob_HG4 = main(n, p, d_list)
np.savetxt("fooL4.csv", prob_L4)
np.savetxt("fooHB4.csv", prob_HB4)
np.savetxt("fooHG4.csv", prob_HG4)

In [266]:
n=5
prob_L5, prob_HB5, prob_HG5 = main(n, p, d_list)
np.savetxt("fooL5.csv", prob_L5)
np.savetxt("fooHB5.csv", prob_HB5)
np.savetxt("fooHG5.csv", prob_HG5)

In [267]:
n=6
prob_L6, prob_HB6, prob_HG6 = main(n, p, d_list)
np.savetxt("fooL6.csv", prob_L6)
np.savetxt("fooHB6.csv", prob_HB6)
np.savetxt("fooHG6.csv", prob_HG6)

In [268]:
n=7
prob_L7, prob_HB7, prob_HG7 = main(n, p, d_list)
np.savetxt("fooL7.csv", prob_L7)
np.savetxt("fooHB7.csv", prob_HB7)
np.savetxt("fooHG7.csv", prob_HG7)

In [269]:
n=8
prob_L8, prob_HB8, prob_HG8 = main(n, p, d_list)
np.savetxt("fooL8.csv", prob_L8)
np.savetxt("fooHB8.csv", prob_HB8)
np.savetxt("fooHG8.csv", prob_HG8)

In [270]:
n=9
prob_L9, prob_HB9, prob_HG9 = main(n, p, d_list)
np.savetxt("fooL9.csv", prob_L9)
np.savetxt("fooHB9.csv", prob_HB9)
np.savetxt("fooHG9.csv", prob_HG9)

In [271]:
n=10
prob_L10, prob_HB10, prob_HG10 = main(n, p, d_list)
np.savetxt("fooL10.csv", prob_L10)
np.savetxt("fooHB10.csv", prob_HB10)
np.savetxt("fooHG10.csv", prob_HG10)

In [279]:
for n in range(11,16):
    #print(n)
    prob_L, prob_HB, prob_HG = main(n, p, d_list)
    file_L = 'fooL' + str(n) + '.csv'
    file_HB = 'fooHB' + str(n) + '.csv'
    file_HG = 'fooHG' + str(n) + '.csv'
    np.savetxt(file_L, prob_L)
    np.savetxt(file_HB, prob_HB)
    np.savetxt(file_HG, prob_HG)
    

In [281]:
for n in range(17,18):
    #print(n)
    prob_L, prob_HB, prob_HG = main(n, p, d_list)
    file_L = 'fooL' + str(n) + '.csv'
    file_HB = 'fooHB' + str(n) + '.csv'
    file_HG = 'fooHG' + str(n) + '.csv'
    np.savetxt(file_L, prob_L)
    np.savetxt(file_HB, prob_HB)
    np.savetxt(file_HG, prob_HG)

In [278]:
'fooL' + str(11) + '.csv'

'fooL11.csv'

In [263]:
#print(prob_L)
#with open('foo.csv','wb') as f_handle:
#    np.savetxt(f_handle,prob_L)